## PyO3

In [1]:
%pip uninstall felsenstein-rs -y
!maturin build --release
%pip install target/wheels/felsenstein_rs-0.1.0-cp311-cp311-manylinux_2_34_x86_64.whl

Found existing installation: felsenstein-rs 0.1.0
Uninstalling felsenstein-rs-0.1.0:
  Successfully uninstalled felsenstein-rs-0.1.0
Note: you may need to restart the kernel to use updated packages.
🔗 Found pyo3 bindings
🐍 Found CPython 3.11 at /home/conda/miniconda3/envs/jax/bin/python3
📡 Using build options features from pyproject.toml
    Finished release [optimized] target(s) in 0.02s
📦 Built wheel for CPython 3.11 to /home/jora/src/rust/felsenstein-rs/target/wheels/felsenstein_rs-0.1.0-cp311-cp311-manylinux_2_34_x86_64.whl
Processing ./target/wheels/felsenstein_rs-0.1.0-cp311-cp311-manylinux_2_34_x86_64.whl
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from itertools import combinations
import felsenstein_rs as frs

In [3]:
# This now gives the correct error message
#ftree_bad = frs.FTree("data/tree_topological.csv", 1e-4)

In [4]:
ftree = frs.FTree("data/tree.txt", 1e-4)

In [5]:
#all_index_pairs = np.array([[*tup] for tup in combinations(range(119), 2)], dtype=np.uint64)

In [6]:
# TODO accept dtype=int instead of uint
index_pairs = np.array([[0, 1], [0, 2], [0, 3]], dtype=np.uint64)
#index_pairs = all_index_pairs

In [7]:
log_p_prior = np.log(np.ones((16,)) * (1/16))
log_p_priors = np.broadcast_to(log_p_prior, (index_pairs.shape[0],) + log_p_prior.shape)

In [8]:
symmetric_matrix = -np.eye(16) + (np.ones((16, 16)) - np.eye(16)) * (1/15)
symmetric_matrices = np.broadcast_to(symmetric_matrix, (index_pairs.shape[0],) + symmetric_matrix.shape)

In [9]:
sqrt_pi = np.random.random(16)**2

In [10]:
sqrt_pi_total = np.broadcast_to(sqrt_pi, (index_pairs.shape[0],) + np.ones(16).shape)

In [11]:
rate_matrix = np.diag(1/sqrt_pi) @ symmetric_matrix @ np.diag(sqrt_pi)
rate_matrices = np.broadcast_to(rate_matrix, (index_pairs.shape[0],) + rate_matrix.shape)

In [12]:
result_old = ftree.infer(index_pairs, rate_matrices, log_p_priors)

In [13]:
result_new = ftree.infer_param(index_pairs, symmetric_matrices, sqrt_pi_total, log_p_priors)

In [14]:
result_old[1][0] - result_new[4][0]

array([[-2.51271226e-13, -3.72080144e-12, -3.38618023e-14,
        -7.43849426e-15,  8.01659086e-16,  2.35922393e-15,
        -2.44804177e-14, -1.30340183e-13, -1.04916076e-14,
        -3.59434704e-15, -8.24895707e-14,  7.79931675e-15,
        -2.25514052e-15,  1.80966353e-13,  8.72635297e-14,
        -1.59872116e-14],
       [-2.56406008e-13, -3.62931907e-12, -8.44879722e-14,
        -9.13713549e-14,  6.81746326e-16, -6.73072709e-16,
        -4.26325641e-14, -2.55129251e-13, -5.12367926e-14,
        -8.49320614e-15, -1.35336187e-13,  1.16573418e-15,
        -2.67667832e-15,  3.57491814e-14, -2.39808173e-14,
        -5.99520433e-14],
       [-5.43676215e-13, -7.86970489e-12,  9.08162434e-14,
        -1.21236354e-13,  1.70870262e-15,  6.80011603e-16,
        -8.22675261e-14, -4.44977388e-13, -8.08242362e-14,
        -1.40720768e-14, -2.47801779e-13,  7.77156117e-15,
        -5.52509427e-15,  1.90958360e-13,  4.44089210e-14,
        -8.29336599e-14],
       [-4.11448653e-13, -6.12576656e